In [ ]:
import pandas as pd
import yfinance as yf

In [ ]:
# Backtesting 500 stocks (S&P500) on an optimized trading strategy and rank stocks based on the overall return
# We buy when the short-term Moving Average (SMA) is above the long-term Moving Average and sell vice-versa

In [ ]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol
tickers = tickers.to_list()

In [ ]:
price_df = yf.download(tickers, start="2010-01-01")

[*********************100%%**********************]  503 of 503 completed

ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2010-01-01 -> 2023-09-18)')


In [ ]:
# We cane slice the DataFrame for a given symbol

In [ ]:
price_df[price_df.columns[price_df.columns.get_level_values(1) == 'AAPL']].head()

,Adj Close,Close,High,Low,Open,Volume
,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2010-01-04 00:00:00,6.487534,7.643214,7.660714,7.585000,7.622500,493729600
2010-01-05 00:00:00,6.498750,7.656429,7.699643,7.616071,7.664286,601904800
2010-01-06 00:00:00,6.395379,7.534643,7.686786,7.526786,7.656429,552160000
2010-01-07 00:00:00,6.383555,7.520714,7.571429,7.466071,7.562500,477131200
2010-01-08 00:00:00,6.425997,7.570714,7.571429,7.466429,7.510714,447610800


In [ ]:
def slice_df(symbol):
  sliced = price_df.copy()
  sliced  = sliced[sliced.columns[sliced.columns.get_level_values(1) == symbol]]
  sliced.columns = sliced.columns.droplevel(1)
  sliced.loc[:,'price'] = sliced.Open.shift(-1) # new price column which contains the Open price from the next day
  return sliced

In [ ]:
def ma_calc(df,n,m): # n > short term window,  m > long term window
  df['sma_1'] = df.Close.rolling(n).mean()
  df['sma_2'] = df.Close.rolling(m).mean()

In [ ]:
test = slice_df('AMZN')
ma_calc(test, 50, 100)
test.tail()

,Adj Close,Close,High,Low,Open,Volume,price,sma_1,sma_2
Date,,,,,,,,,
2023-09-11 00:00:00,143.100006,143.100006,143.619995,138.639999,138.750000,56764500,142.320007,133.864001,125.236001
2023-09-12 00:00:00,141.229996,141.229996,143.000000,140.610001,142.320007,42668500,140.949997,134.081401,125.605301
2023-09-13 00:00:00,144.850006,144.850006,144.979996,140.869995,140.949997,60465200,145.080002,134.374001,126.015701
2023-09-14 00:00:00,144.720001,144.720001,145.860001,142.949997,145.080002,64033600,142.690002,134.660801,126.393301
2023-09-15 00:00:00,140.389999,140.389999,143.570007,140.089996,142.690002,102861700,NaN,134.901401,126.735101


In [ ]:
def backtest(df, n, m):
    ma_calc(df,n,m)
    in_position = False
    profits = []

    for index,row in df.iterrows():
      if not in_position:
          if row.sma_1 > row.sma_2:
            #print('Buy:')
            #print(index)
            buyprice = row.price
            in_position = True
      if in_position:
        if row.sma_1 < row.sma_2:
          #print('Buy:')
          #print(index)
          profit = (row.price - buyprice)/buyprice
          profits.append(profit)
          in_position = False

    gain = (pd.Series(profits) + 1).prod()
    return gain

In [ ]:
backtest(test,50,100)
# one dollar would be roughly 4.90 dollars

4.905698876447606

In [ ]:
# Now we want to apply it to all symbols

In [ ]:
results = []

for symbol in tickers:
  subdf = slice_df(symbol)
  results.append(backtest(subdf,50,100))

'''result for BG:
1.2833572850247978
result for CHRW:
0.7068334009172945
result for CDNS:
2.7961580725226725
result for CZR:
1.0532426473065881
result for CPT:
1.5180080395573912
result for CPB:
0.9081260131836466
result for COF:
1.828958725188113
...'''

In [ ]:
profits = pd.DataFrame({'profit':results},index=tickers)
profits.head()

,profit
MMM,1.482391
AOS,3.685766
ABT,1.392583
ABBV,1.393048
ACN,2.425557


In [ ]:
profits.nlargest(10,'profit')

,profit
NVDA,47.337730
TSLA,40.964832
NFLX,29.981966
ENPH,26.821747
DXCM,23.048400
ETSY,16.317201
MU,15.507921
ODFL,10.282048
MKTX,9.893758
URI,9.553678


In [ ]:
profits.nsmallest(10,'profit')

,profit
APA,0.050377
OXY,0.169404
PCG,0.251056
WBD,0.344487
CCL,0.349183
CTLT,0.368093
EIX,0.413298
WDC,0.435257
HAL,0.445801
KDP,0.459292


In [ ]:
# VECTORIAL APPROACH

In [ ]:
test.sma_1 > test.sma_2 # buy signal

In [ ]:
(test.sma_1 > test.sma_2).idmax() # the first occurrence (date) of the buy signal

In [ ]:
first_buy = pd.Series(test.index == (test.sma_1 > test.sma_2).idxmax(), index=test.index)

In [ ]:
True + True

2

In [ ]:
True + False

1

In [ ]:
X_ = pd.Series([False,False,True,True])
X_.diff()

0      NaN
1    False
2     True
3    False
dtype: object

In [ ]:
real_signal = first_buy | (test.sma_1 > test.sma_2).diff()

In [ ]:
trades = test[real_signal]

In [ ]:
# The rows are now alternated between buying rows and selling rows

In [ ]:
if (len(trades) %2) != 0:
  mtm = test.tail(1).copy()
  mtm.price = mtm.Close
  trades = pd.concat([trades,mtm])

profits = trades.price.diff()[1::2] / trades.price[0::2].values
gain = (profits +1).prod()

In [ ]:
def backtest(df, n, m):
    ma_calc(df,n,m)
    in_position = False
    profits = []

    for index,row in df.iterrows():
      if not in_position:
          if row.sma_1 > row.sma_2:
            #print('Buy:')
            #print(index)
            buyprice = row.price
            in_position = True
      if in_position:
        if row.sma_1 < row.sma_2:
          #print('Buy:')
          #print(index)
          profit = (row.price - buyprice)/buyprice
          profits.append(profit)
          in_position = False

    if in_position:
      profit = (row.Close - buyprice) / buyprice
      profits.append(profit)

    gain = (pd.Series(profits) + 1).prod()
    return gain

In [ ]:
backtest(test,50,100)

7.38801816403885

In [ ]:
def vectorized(df,n,m):
  ma_calc(df,n,m)
  first_buy = pd.Series(test.index == (test.sma_1 > test.sma_2).idxmax(), index=test.index)
  real_signal = first_buy | (test.sma_1 > test.sma_2).diff()
  trades = df[real_signal]
  if (len(trades)%2) != 0:
    mtm = test.tail(1).copy()
    mtm.price = mtm.Close
    trades = pd.concat([trades,mtm])
    profits = trades.price.diff()[1::2] / trades.price.diff()[0::1].values
    gain = (profits +1).prod()
    return gain